In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob
import os
sys.path.insert(0,"/home/daqtest/Processor/sandpro")
import sandpro
import configparser
import json
import scipy.stats
from scipy.optimize import curve_fit
import datetime
import pandas as pd
# %run ../WaveformProcessor.py
# %run ../FastProcessing.py
# %run ../FitSPE.py

sys.path.insert(0,"../src/")
import common.d2d as d2d
import common.utils as util
import data_processing.fast_processor as fast_processor

### Check which datasets are available

In [ ]:
# path = "/home/daqtest/DAQ/SandyAQ/softlink_to_data/noise_time_evolution_before_threshold_calibration_implemented/"
# path = "/home/daqtest/DAQ/SandyAQ/softlink_to_data/all_data/20240819_T102_50V_5.0sig/"
path = "/home/daqtest/DAQ/SandyAQ/softlink_to_data/all_data/20240624_T100_46V_4.0sig/"
# path = "/home/daqtest/DAQ/SandyAQ/softlink_to_data/all_data/T100_47_5,5sig_V/" # check waveform
calibration_run = False # True if you want to check the calibration, False otherwise
plot = False # if want to replot the waveforms
# ignore_channel_list = np.array([2,8,11,14,20,23])
ignore_channel_list = np.array([])

# ##########################
# Load the meta data files
if calibration_run:
    data_folder = os.path.join(path,"threshold_calibration/")
else:
    data_folder = path

meta_data_list = glob.glob(f"{data_folder}/*meta*")
#reorganize the order of the meta_data_list, so the final one should be the newest one.

# Sort the file paths based on the extracted date and time in ascending order (oldest to newest)
new_meta_data_list = sorted(meta_data_list, key=util.extract_date_meta_data)
new_meta_data_list = sorted(new_meta_data_list[-24:], key=util.extract_channel_meta_data)
meta_data_list = new_meta_data_list[-24:]
meta_data_list

Trying to remove the PostProcessing legacy


In [ ]:
FastProcessor = fast_processor.FastProcessor(data_folder, meta_data_list, 
               ignore_channel_list=ignore_channel_list,
               calibration_run=calibration_run)
FastProcessor.process_runs()


In [ ]:
plt.close()

FastProcessor.plot_waveforms(save_plot = False, show_plot=True)


### Check waveform

In [ ]:
# path = "/home/daqtest/DAQ/SandyAQ/softlink_to_data/noise_time_evolution_before_threshold_calibration_implemented/"
# path = "/home/daqtest/DAQ/SandyAQ/softlink_to_data/all_data/20240628_2_T102_47V_6.0sig/"
path = "/home/daqtest/DAQ/SandyAQ/softlink_to_data/all_data/T100_47_5,5sig_V/" # check waveform
calibration_run = False # True if you want to check the calibration, False otherwise
plot = False # if want to replot the waveforms
ignore_channel_list = np.array([2,8,11,14,20,23])

# ##########################
# Load the meta data files
if calibration_run:
    data_folder = os.path.join(path,"threshold_calibration/")
else:
    data_folder = path

meta_data_list = glob.glob(f"{data_folder}/*meta*")
#reorganize the order of the meta_data_list, so the final one should be the newest one.

# Sort the file paths based on the extracted date and time in ascending order (oldest to newest)
new_meta_data_list = sorted(meta_data_list, key=util.extract_date_meta_data)
new_meta_data_list = sorted(new_meta_data_list[-24:], key=util.extract_channel_meta_data)
meta_data_list=new_meta_data_list[-24:]

post_processing = FastProcessing(data_folder, meta_data_list[3], 
               ignore_channel_list=ignore_channel_list,
               calibration_run=calibration_run)

spd =  post_processing.process_run()

df = post_processing.df

randomly_selected_raw_WF = np.transpose(post_processing.df.randomly_selected_raw_WF[0])
randomly_selected_filtered_WF = np.transpose(post_processing.df.randomly_selected_filtered_WF[0])
baseline_mean = df.baseline_mean[0]
data_end = 500
raw_WF = randomly_selected_raw_WF[:data_end,20]
filtered_WF = randomly_selected_filtered_WF[:data_end,20]
time = np.arange(0, data_end, 1) * 4
plt.plot(time,raw_WF-baseline_mean)
plt.plot(time,filtered_WF)

In [ ]:
df = pd.DataFrame({"time[ns]":time,
                   "raw_waveform[V]":raw_WF})

In [ ]:
df.to_csv("raw_waveform_zepeng.csv")

In [ ]:
df = post_processing.df

In [ ]:
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

In [ ]:

hist_count, bin_edges, _ = plt.hist(df[df["channel"]==13].areas,bins=200,range=(-0.1,10),histtype='step',color='red')
bin_centers = bin_edges[:-1] + np.diff(bin_edges)/2

bin_density = 10/len(bin_centers)
distance_rough_guess = 0.5 #unit
peaks, _ = find_peaks(hist_count, height=5, distance=distance_rough_guess/bin_density)
# peaks, _ = find_peaks(n_hist, height=5)

plt.plot(bin_centers,hist_count,color='red',label='channel 1')
plt.plot(bin_centers[peaks], hist_count[peaks], "x")


In [ ]:
spe_fit = FitSPE.FitSPE(hist_count, bin_edges, plot = False, show_plot=False, save_plot=False)
np.transpose(spe_fit.line_x)[:,np.array(spe_fit.mu_err_list)<0.02].shape




In [ ]:
%run ../FitSPE.py


In [ ]:
test = FitSPE(n_hist, bin_edge, plot=True)


In [ ]:
PE_rough_position = bin_centers[peaks] #unit: mV*ns
PE_rough_half_width = np.median(np.diff(PE_rough_position))/2
PE_rough_amplitude = n_hist[peaks]
PE_half_width_index = int(np.median(np.diff(peaks))/2) # PE width in index


In [ ]:
# Let's create a function to model and create data 
def func(x, a, x0, sigma): 
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

fit_which_peak = 0
# PE_width_n = int(np.median(np.diff(peaks))/2)
# peak=peaks[fit_which_peak]

min_x_list = []
max_x_list = []

plt.plot(bin_centers, n_hist, c='black', label='data') 


# Executing curve_fit on noisy data 

for i, peak in enumerate(peaks[0:10]):
    min_x = int(peak-PE_half_width_index)
    max_x = int(peak+PE_half_width_index)
    min_x_list.append(min_x)
    max_x_list.append(max_x)
    
    (amp,mu,sig), pcov = curve_fit(func, bin_centers[min_x:max_x], n_hist[min_x:max_x], p0=[PE_rough_amplitude[i], PE_rough_position[i], PE_rough_half_width])
  
    ym = func(bin_centers[min_x:max_x], amp,mu,sig) 
    plt.plot(bin_centers[min_x:max_x], ym, c='r', label='Best fit') 


In [ ]:
PE_rough_half_width

In [ ]:
def fit_histograms(self, n_hist, bin_edges, plot=True):
        
        bin_centers = bin_edges[:-1] + np.diff(bin_edges)/2
        bin_density = 10/len(bin_centers)
        distance_rough_guess = 0.5 # distance between peaks in mV*ns

        peaks, _ = find_peaks(n_hist, height=5, distance=distance_rough_guess/bin_density)

        PE_rough_position = bin_centers[peaks] # unit: mV*ns
        PE_rough_half_width = np.median(np.diff(PE_rough_position))/2
        PE_rough_amplitude = n_hist[peaks]
        PE_half_width_index = int(np.median(np.diff(peaks))/2) # PE width in index
        
        # Executing curve_fit on noisy data 
        for peak in peaks:
            min_x = peak - PE_half_width_index
            max_x = peak + PE_half_width_index
            (amp,mu,sig), _ = curve_fit(self._gaussian_func, 
                                bin_centers[min_x:max_x], 
                                n_hist[min_x:max_x], 
                                p0=[PE_rough_amplitude[fit_which_peak], 
                                    PE_rough_position[fit_which_peak], 
                                    PE_rough_half_width])
        
        #popt returns the best fit values for parameters of the given model (func) 
        print (amp,mu,sig) 
        
        ym = self._gaussian_func(bin_centers, amp, mu, sig) 
        plt.plot(bin_centers, n_hist, c='black', label='data') 
        plt.plot(bin_centers, ym, c='r', label='Best fit') 

        if plot:
            plt.figure()
            plt.plot(bin_centers,n_hist,color='red',label='channel 1')
            plt.plot(bin_centers[peaks], n_hist[peaks], "x")
            plt.show()

def _gaussian_func(x, a, x0, sigma): 
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

### Loop for all channels

In [ ]:
# # run_id = "1" # name for output file

# # last_n_dataset = 2
# # sorted_meta_data_list = meta_data_list[-24*last_n_dataset-1:-24*last_n_dataset+23]
# sorted_meta_data_list = meta_data_list[-24:]

# ignore_channel_list = np.array([2,8,11,14,20,23])
# # for the ignore channel list, we ignore 2,8,11,14,20,23. 
# # I add a few channels in the list cuz I wanna to do some coding  to #3.

# #plot settings
# fontsize = "small"

# baseline_mean_list = []
# baseline_std_list = []
# mu_all_channel=[]
# datetime_list = []
# bias_voltage_list = []

# for i in range(len(sorted_meta_data_list)):
#     print(sorted_meta_data_list[i])
#     meta_data = sorted_meta_data_list[i]
#     meta_data_basename = os.path.basename(meta_data)
#     parts = meta_data_basename.split('_')
    
#     config_name = "_".join(parts[1:4]) + ".ini"
#     threshold_adc = parts[3]
#     channel = int(parts[2])
#     date_datafile = int(parts[4])
#     time_datafile = int(parts[5].split('.')[0])

#     datetime_object = datetime.datetime.strptime(f"{parts[4]} {parts[5].split('.')[0]}", '%Y%m%d %H%M%S')
#     datetime_list.append(datetime_object)

#     # ignore the channels that in the ignore_channel_list
#     if channel in ignore_channel_list:
#         continue

#     # read from meta data file
#     with open(meta_data, "r") as f:
#         data_taking_settings = json.load(f)

#     n_events = int(data_taking_settings["number_of_events"])
#     bias_voltage = float(data_taking_settings["voltage_config"]["preamp_1"]) # assuming all preamp has the same bias voltage; can be easily changed
#     bias_voltage_list.append(bias_voltage)

#     print(config_name)
#     config_path = os.path.join(data_folder, "tmp",config_name)
#     config = configparser.ConfigParser()
#     config.optionxform = str
#     config.read(config_path)

#     process_config = {"nchs": 1,
#     "nsamps": int(config.get("COMMON", "RECORD_LENGTH")),
#     "sample_selection": 120,
#     "samples_to_average": 40}

#     # dump the config to a json file
#     with open("process_config.json", "w") as f:
#         json.dump(process_config, f)
#         # set the board number and integral window according to the board number (board 0 and 1 have different integral windows)
#     if len(config.get("BOARD-0", "CHANNEL_LIST")) > 0:
#         board_number = 0
#         local_channel = channel
#         integral_window = (0.3,0.55)
#     else:
#         board_number = 1
#         integral_window = (0.4,0.6)
#         local_channel = channel - 16

#     processor= sandpro.processing.rawdata.RawData(config_file = "process_config.json",
#                                                perchannel=False)
#     data_file_basename = meta_data_basename.replace("meta_", "").replace(".json", f"_board_{board_number}.bin")

#     # section_name = f"BOARD-{board_number}_CHANNEL-{local_channel}"
#     # option_name = "N_EVENTS"
#     # if config.has_option(section_name, option_name):
#     # if n_events > :
#         # n_events = int(config.get(section_name, option_name))
#         # data = processor.get_rawdata_numpy(n_evts=int(n_events)-1,
#         #                             file=os.path.join(data_folder, data_file_basename),
#         #                             bit_of_daq=14,
#         #                             headersize=4,inversion=False)
#         # # start_index, end_index = 1500, 1980
#         # start_index, end_index = 1000, int(n_events)-1
#         # print(f"analysing events from {start_index} to {end_index}")
#     # else:
#     try:
#         data = processor.get_rawdata_numpy(n_evts=n_events-1,
#                                     file=os.path.join(data_folder, data_file_basename),
#                                     bit_of_daq=14,
#                                     headersize=4,inversion=False)
#         # start_index, end_index = 1500, 1980
#         start_index, end_index = 1000, n_events-1-500
#         print(f"analysing events from range: {start_index} to {end_index}")
#     except Exception as e:
#         print(e)
#         data = processor.get_rawdata_numpy(1999,
#                                     file=os.path.join(data_folder, data_file_basename),
#                                     bit_of_daq=14,
#                                     headersize=4,inversion=False)
#         start_index, end_index = 1000, 1999


#     wfp = WFProcessor(data_folder, volt_per_adc=2/2**14)
#     wfp.set_data(data["data_per_channel"][start_index:end_index,0], in_adc = False)
#     wfp.process_wfs()
    
#     baseline_std = np.mean(wfp.baseline_rms)
#     baseline_mean = np.mean(wfp.baseline)
#     baseline_mean_list.append(baseline_mean)
#     baseline_std_list.append(baseline_std)
#     n_processed_events = len(wfp.baseline_rms)

#     # print("number of events: ", len(wfp.baseline_rms))

#     threshold_mv = util.adc_to_mv(int(threshold_adc))
#     # set the threshold to be 2 * Vpp
#     optimal_threshold_mv =  (baseline_mean + 2 * (2 * np.sqrt(2) * baseline_std)) * 1000
#     optimal_threshold_3sig_mv =  (baseline_mean + 3 * baseline_std) * 1000
#     optimal_threshold_adc = util.mv_to_adc(optimal_threshold_mv)
#     areas = wfp.get_area(sum_window=integral_window)
#     heights = wfp.get_height(search_window=integral_window)

#     print("thresholds, opt, thres: ",optimal_threshold_adc, threshold_adc)

#     def gaussian(x, amplitude, mean, sigma):
#         return amplitude * np.exp(-((x - mean) / sigma) ** 2 / 2)
#     # we will use the gaussian fit later

#     if plot:
#         figure, axes = plt.subplots(3,2,figsize=(15,15))
#         # figure.subplots_adjust(wspace=0.5)
#         # figure.subplots_adjust(hspace=0.5)
#         time = np.arange(0, process_config["nsamps"], 1) * 4

#         data_processed = data["data_per_channel"][start_index:end_index,0,:]
#         mask = np.random.rand(n_processed_events) < 0.05
#         data_processed[mask,:]
#         axes[0,0].plot(time, 1000 * np.transpose(data_processed[mask,:]),color="red",alpha=0.2)
#         axes[1,0].plot(time, 1000 * np.transpose(data_processed[mask,:]),color="red",alpha=0.2)
#         i -= start_index
#         axes[0,1].plot(time, 1000 * np.transpose(wfp.filtered_wfs[mask,:]),color="red",alpha=0.2)
#         axes[1,1].plot(time, 1000 * np.transpose(wfp.filtered_wfs[mask,:]),color="red",alpha=0.2)
        
#         # forloop is slower
#         # for i in range(start_index, end_index):
#         #     # print 5% random waveforms within the index
#         #     if np.random.rand() < 0.05:
#         #         axes[0,0].plot(time, 1000 * data["data_per_channel"][i][0],color="red",alpha=0.2)
#         #         axes[1,0].plot(time, 1000 * data["data_per_channel"][i][0],color="red",alpha=0.2)
#         #         i -= start_index
#         #         axes[0,1].plot(time, 1000 * wfp.filtered_wfs[i],color="red",alpha=0.2)
#         #         axes[1,1].plot(time, 1000 * wfp.filtered_wfs[i],color="red",alpha=0.2)
#         axes[0,0].axhline(optimal_threshold_mv, color='g',label=f"Optimal threshold: \n{int(optimal_threshold_adc)}[ADC] \n{int(optimal_threshold_mv)}[mV]")
#         axes[0,0].axhline(optimal_threshold_3sig_mv, color='g',label=f"3 sig threshold: \n{int(optimal_threshold_mv)}[mV]")
#         axes[0,0].axhline(threshold_mv, color='b',label=f"Test threshold: \n{int(threshold_adc)}[ADC] \n{int(threshold_mv)}[mV]", zorder=10)
#         axes[0,0].set_xlim(0,3900)

#         axes[1,0].axhline(optimal_threshold_mv, color='g',label=f"Optimal threshold: \n{int(optimal_threshold_adc)}[ADC] \n{int(optimal_threshold_mv)}[mV]")
#         axes[1,0].axhline(optimal_threshold_3sig_mv, color='g',label=f"3 sig threshold: \n{int(optimal_threshold_mv)}[mV]")
#         axes[1,0].axhline(threshold_mv, color='b',label=f"Test threshold: \n{int(threshold_adc)}[ADC] \n{int(threshold_mv)}[mV]", zorder=10)
#         axes[1,0].set_xlim(0,3900)

#         _ymax = 1800
#         axes[0,0].set_ylim(200,_ymax)
#         axes[0,0].text(100,optimal_threshold_mv + _ymax/5,f"baseline mean: {1000 * baseline_mean:.1f} mV")
#         axes[0,0].text(100,optimal_threshold_mv + _ymax/10,f"baseline std: {1000 * baseline_std:.2f} mV")
#         axes[0,0].set_ylabel("Voltage [mV]",fontsize=fontsize)
#         axes[0,0].set_xlabel("ADC sample [ns]",fontsize=fontsize)
#         # plot intergral window
#         axes[0,0].fill_betweenx([200,_ymax], integral_window[0]*np.max(time), integral_window[1]*np.max(time), color='gray', alpha=0.5)

#         axes[0,1].set_xlim(0,3900)
#         axes[0,1].set_ylim(-5,1600)
#         axes[0,1].set_ylabel("Voltage [mV]",fontsize=fontsize)
#         axes[0,1].set_xlabel("ADC sample [ns]",fontsize=fontsize)
        
#         _ymax = 350
#         axes[1,0].set_xlim(0,3900)
#         axes[1,0].set_ylim(200,_ymax)
#         axes[1,0].text(100,optimal_threshold_mv + _ymax/20,f"baseline mean: {1000 * baseline_mean:.1f} mV")
#         axes[1,0].text(100,optimal_threshold_mv + _ymax/40,f"baseline std: {1000 * baseline_std:.2f} mV")
#         axes[1,0].set_ylabel("Voltage [mV]",fontsize=fontsize)
#         axes[1,0].set_xlabel("ADC sample [ns]",fontsize=fontsize)
#         # plot intergral window
#         axes[1,0].fill_betweenx([200,_ymax], integral_window[0]*np.max(time), integral_window[1]*np.max(time), color='gray', alpha=0.5)
#         axes[1,0].axhline(1000 * baseline_mean, color="gray",linestyle="dashed",label=f"Baseline mean", zorder=10)


#         _ymax = 40
#         axes[1,1].set_xlim(0,3900)
#         axes[1,1].set_ylim(-5,_ymax)
#         axes[1,1].set_ylabel("Voltage [mV]",fontsize=fontsize)
#         axes[1,1].set_xlabel("ADC sample [ns]",fontsize=fontsize)

#         hist,bin_edges,_ = axes[2,0].hist(areas,bins=200,range=(-0.1,10),histtype='step',color='red')
#         axes[2,0].set_yscale("log")
#         axes[2,0].set_ylabel("Count",fontsize=fontsize)
#         axes[2,0].set_xlabel("Integrated Area [$V\cdot ns$]",fontsize=fontsize)
#         axes[2,0].set_ylim(1e-1,None)

#         # axes[1,1].hist(heights,bins=100,range=(0,40),histtype='step')
#         # axes[2,1].hist2d(areas,heights,bins=[200,100],range=[[-0.1,10],[0,40]],cmap='viridis',norm="log")
#         axes[2,1].hist2d(areas,heights,bins=[200,100],cmap='viridis',norm="log")
#         axes[2,1].set_ylabel("Filtered Height [mV]",fontsize=fontsize)
#         axes[2,1].set_xlabel("Filtered integrated area [$V\cdot ns$]",fontsize=fontsize)
        
#         for ax in axes.flatten():
#             ax.legend()
#         axes[0,0].set_title(f"Raw WFs channel {channel}")
#         axes[0,1].set_title(f"Filtered WFs channel {channel}")


#         # if not calibration_run:
#         #     ##########finding the peak, and the specifit range###############
#         #     hist_diff = np.diff(hist)  # Compute differences between consecutive histogram bins
#         #     peaks = np.where((hist_diff[:-1] > 0) & (hist_diff[1:] < 0))[0] + 1  # Find indices where difference changes sign

#         #     peak_indices = []
#         #     hist_index=[]
#         #     for peak_index in peaks:
#         #         peak_indices.append(peak_index)
#         #         hist_index.append(hist[peak_index])
#         #         #axes[2, 0].plot(bin_edges[peak_index], hist[peak_index], marker='o', color='blue', markersize=8, label='Peak')


#         #     edge1 = (bin_edges[peak_indices[0]]+bin_edges[peak_indices[1]])/2
#         #     edge2 = (bin_edges[peak_indices[1]]+bin_edges[peak_indices[2]])/2
#         #     edge3 = (bin_edges[peak_indices[2]]+bin_edges[peak_indices[3]])/2
#         #     edge=[edge1,edge2,edge3]
        
            
#         #     bin_width = bin_edges[1] - bin_edges[0]

#         #     for k in edge:
#         #         bin_index = int((k- bin_edges[0]) / bin_width)
#         #         corresponding_count = hist[bin_index]
#         #         axes[2,0].plot(k, corresponding_count, marker='o', color='blue', markersize=6, label='Peak')
            
        
        
#         #     ##########adding the gaussian fit###############
#         #     # Define the range of the specific part of the data you want to use for fitting
#         #     pe=[]
#         #     for j in range(len(edge)-1):
#         #         specific_part_range = (edge[j],edge[j+1])
#         #         specific_part_areas = [area for area in areas if specific_part_range[0] <= area <= specific_part_range[1]]
#         #         mu, sigma = scipy.stats.norm.fit(specific_part_areas)
#         #         bins=np.linspace(-0.1, 10, 201)
#         #         bins_subset = bins[(bins >= specific_part_range [0]) & (bins <= specific_part_range [1])]
                
#         #         axes[2, 0].plot(bins_subset, scipy.stats.norm.pdf(bins_subset, mu, sigma), color='green', label='Gaussian Fit')

#         #         min_y, max_y = axes[2, 0].get_ylim()
#         #         axes[2, 0].vlines(mu, min_y, max_y, color='green', label='Gaussian Fit')

#         #         pe.append(mu) # this line has the information of pe1 and pe2 for the gaussian fit

#         #     ############ normalize it################
#         #     # specific_hist, specific_bin_edges = np.histogram(specific_part_areas, bins=200, range=(edge1, edge2))

        
#         #     # norm=np.sqrt(sum([i**2 for i in specific_part_areas]))
#         #     # normalized=specific_hist/norm

#         #     # bin_width = specific_part_areas[1] - specific_part_areas[0]

#         #     # # Create the bar plot for the histogram
#         #     # axes[2,0].bar(edge2, specific_hist, width=bin_width, edgecolor='black', alpha=0.7)
        
#         #     #################################################################################################
        
#         # mu_all_channel.append(pe)


#         figure_path = os.path.join(data_folder,"plot/")
#         if not os.path.exists(figure_path):
#                 os.makedirs(figure_path)

#         plt.legend(fontsize="small")
#         plt.savefig(os.path.join(figure_path,f"plot_{channel}_{date_datafile}_{time_datafile}.png"))
    
#         plt.show()

In [ ]:
hist_diff = np.diff(hist)  # Compute differences between consecutive histogram bins
peaks = np.where((hist_diff[:-1] > 0) & (hist_diff[1:] < 0))[0] + 1  # Find indices where difference changes sign

plt.plot(hist)
plt.plot(hist_diff)
plt.plot(bin_edges[:-1][peaks],hist[peaks],"o")

# peak_indices = []
# hist_index=[]
# for peak_index in peaks:
#     peak_indices.append(peak_index)
#     hist_index.append(hist[peak_index])
#     #axes[2, 0].plot(bin_edges[peak_index], hist[peak_index], marker='o', color='blue', markersize=8, label='Peak')


# edge1 = (bin_edges[peak_indices[0]]+bin_edges[peak_indices[1]])/2
# edge2 = (bin_edges[peak_indices[1]]+bin_edges[peak_indices[2]])/2
# edge3 = (bin_edges[peak_indices[2]]+bin_edges[peak_indices[3]])/2
# edge=[edge1,edge2,edge3]


# bin_width = bin_edges[1] - bin_edges[0]

# for k in edge:
#     bin_index = int((k- bin_edges[0]) / bin_width)
#     corresponding_count = hist[bin_index]
#     axes[2,0].plot(k, corresponding_count, marker='o', color='blue', markersize=6, label='Peak')



# ##########adding the gaussian fit###############
# # Define the range of the specific part of the data you want to use for fitting
# pe=[]
# for j in range(len(edge)-1):
#     specific_part_range = (edge[j],edge[j+1])
#     specific_part_areas = [area for area in areas if specific_part_range[0] <= area <= specific_part_range[1]]
#     mu, sigma = scipy.stats.norm.fit(specific_part_areas)
#     bins=np.linspace(-0.1, 10, 201)
#     bins_subset = bins[(bins >= specific_part_range [0]) & (bins <= specific_part_range [1])]
    
#     axes[2, 0].plot(bins_subset, scipy.stats.norm.pdf(bins_subset, mu, sigma), color='green', label='Gaussian Fit')

#     min_y, max_y = axes[2, 0].get_ylim()
#     axes[2, 0].vlines(mu, min_y, max_y, color='green', label='Gaussian Fit')

#     pe.append(mu) # this line has the information of pe1 and pe2 for the gaussian fit
# print("heyyyyyyyyyyy pe",pe)

### Noise Level Time Evolution

In [ ]:
path = "/home/daqtest/DAQ/SandyAQ/softlink_to_data/noise_time_evolution_before_threshold_calibration_implemented/"
# path = "/home/daqtest/DAQ/SandyAQ/softlink_to_data/*"
calibration_run = False # True if you want to check the calibration, False otherwise
plot = False # if want to replot the waveforms
ignore_channel_list = np.array([2,8,11,14,20,23])

# ##########################
# Load the meta data files
if calibration_run:
    data_folder = os.path.join(path,"threshold_calibration/")
else:
    data_folder = path

meta_data_list = glob.glob(f"{data_folder}/*meta*202405")
#reorganize the order of the meta_data_list, so the final one should be the newest one.

# Sort the file paths based on the extracted date and time in ascending order (oldest to newest)
new_meta_data_list = sorted(meta_data_list, key=util.extract_date_meta_data)
new_meta_data_list = sorted(new_meta_data_list[:], key=util.extract_channel_meta_data)
meta_data_list=new_meta_data_list
len(meta_data_list)

In [ ]:
post_processing = PostProcessing(data_folder, meta_data_list, 
               ignore_channel_list=ignore_channel_list,
               calibration_run=calibration_run)

spd = post_processing.process_run(make_plot=False, show_plot=False)

In [ ]:
df = post_processing.df

In [ ]:
type(df.loc[0])

In [ ]:
bias_voltage = np.unique(df.bias_voltage.values)
df.bias_voltage.values

In [ ]:
# rows = df.loc[df['bias_voltage'] == -48.]
# plt.plot(rows['datetime_obj'].values, rows['baseline_std'].values, 'o')

In [ ]:
figure, (ax1, ax2)= plt.subplots(2,1, figsize=(10, 6))
ax1.set_ylabel("baseline std")
ax2.set_ylabel("baseline mean")


plt.xticks(rotation=45)


for voltage in bias_voltage:
    rows = df.loc[df['bias_voltage'] == voltage]
    ax1.plot(rows['datetime_obj'].values, rows['baseline_std'].values, 'o', label=str(voltage))
    
    # color_baseline = "tab:red"
    # ax.plot(datetime_list[mask], baseline_mean_list[mask], 'o--', label=str(voltage), color=color_baseline)
    # ax.set_ylabel("baseline", color=color_baseline)
    # ax.tick_params(axis='y', labelcolor=color_baseline)

    ax2.plot(rows['datetime_obj'].values, rows['baseline_mean'].values, 'o', label=str(voltage))

    # color_rms = "tab:blue"
    # ax2.plot(datetime_list[mask], baseline_std_list[mask], 'o-', label=str(voltage), color=color_rms)
    # ax2.set_ylabel("baseline std", color=color_rms)
    # ax2.tick_params(axis='y', labelcolor=color_rms)

ax1.plot(df['datetime_obj'].values, df['baseline_std'].values, '-')
ax2.plot(df['datetime_obj'].values, df['baseline_mean'].values, '--')


plt.legend(fontsize="small")


plt.show()

    

In [ ]:
figure, (ax1, ax2)= plt.subplots(2,1, figsize=(10, 6),sharex=True)

ax1.set_ylabel("baseline std")
ax2.set_ylabel("baseline mean")


ax1.set_xlabel("threshold")


plt.xticks(rotation=45)


for voltage in bias_voltage:
    rows = df.loc[df['bias_voltage'] == voltage]
    ax1.scatter(rows['threshold_adc'].values.astype(float), rows['baseline_std'].values, marker = 'o', label=str(voltage))
    ax2.scatter(rows['threshold_adc'].values.astype(float), rows['baseline_mean'].values, marker = 'o', label=str(voltage))

plt.legend(fontsize="small")


plt.show()

    

In [ ]:
df.corr(numeric_only=True)

### Extract the config name, and fing the config in the tmp folder

In [ ]:
meta_data = meta_data_list[-3]
print(meta_data[0])# makesure it is the newest one
meta_data_basename = os.path.basename(meta_data)
parts = meta_data_basename.split('_')
print(parts)
config_name = "_".join(parts[1:4]) + ".ini"
threshold_adc = parts[3]
channel = int(parts[2])
time = int(parts[5].split('.')[0])

print(config_name)
config_path = os.path.join(data_folder, "tmp",config_name)
config = configparser.ConfigParser()
config.optionxform = str
config.read(config_path)

### Set the processor config

In [ ]:
process_config = {"nchs": 1,
 "nsamps": int(config.get("COMMON", "RECORD_LENGTH")),
 "sample_selection": 120,
 "samples_to_average": 40
}

# dump the config to a json file
with open("process_config.json", "w") as f:
    json.dump(process_config, f)
if len(config.get("BOARD-0", "CHANNEL_LIST")) > 0:
    board_number = 0
else:
    board_number = 1

In [ ]:
a = np.array([1,2,3,4])
a<3

### Load Data and Process waveform

In [ ]:
processor= sandpro.processing.rawdata.RawData(config_file = "process_config.json",
                                               perchannel=False)
data_file_basename = meta_data_basename.replace("meta_", "").replace(".json", f"_board_{board_number}.bin")
data_file_basename

In [ ]:
data = processor.get_rawdata_numpy(n_evts=4999,
                                   file=os.path.join(data_folder, data_file_basename),
                                   bit_of_daq=14,
                                   headersize=4,inversion=False)

In [ ]:
start_index, end_index = 1, 4999

wfp = WFProcessor(data_folder, volt_per_adc=2/2**14)
wfp.set_data(data["data_per_channel"][start_index:end_index,0], in_adc=False)

In [ ]:
wfp.process_wfs()
wfp.plot_random_wfs(5,filtered=True)

In [ ]:
heights = wfp.get_height(search_window=(0.2,0.8))
areas = wfp.get_area(sum_window=(0.2,0.8))
plt.hist(areas, bins=np.linspace(0, 10, 200))
plt.yscale('log')
plt.show()

### Calculate optimal threshold

In [ ]:
# def adc_to_mv(adc, DCOFFSET=+40, vpp=2.0, bit_of_daq=14):
#     #DC OFFSET = +40: -0.2 - 1.8
#     #DC OFFSET = +50: 0.0 - 2.0
#     #DC OFFSET = -50: -2 - 0.0
#     start_voltage = (DCOFFSET/50) -1
#     # end_voltage = start_voltage + vpp
#     voltage_per_adc = vpp / (2**bit_of_daq-1)
#     # return (adc/(2**bit_of_daq-1) * vpp + start_voltage) * 1000
#     return (adc * voltage_per_adc + start_voltage) * 1000

# def mv_to_adc(mv, DCOFFSET=+40, vpp=2.0, bit_of_daq=14):
#     start_voltage = (DCOFFSET/50) -1
#     # end_voltage = start_voltage + vpp
#     voltage_per_adc = vpp / (2**bit_of_daq-1)
#     # return (mv/1000 - start_voltage) / vpp * (2**bit_of_daq-1)
#     return (mv/1000 - start_voltage) / voltage_per_adc

In [ ]:
# Data per channel is in volts
# Need to convert the threshold to volts

# Baseline std and baseline mean are also in volts
baseline_std = np.mean(wfp.baseline_rms)
baseline_mean = np.mean(wfp.baseline)

threshold_mv = util.adc_to_mv(int(threshold_adc))

# set the threshold to be 2 * Vpp
optimal_threshold_mv = (baseline_mean + 2 * (2 * np.sqrt(2) * baseline_std)) * 1000

optimal_threshold_adc = mv_to_adc(optimal_threshold_mv)


time = np.arange(0, process_config["nsamps"], 1) * 4
for i in range(start_index,end_index):
    plt.plot(time, data["data_per_channel"][i][0] * 1000)
plt.axhline(optimal_threshold_mv, color='g',label=f"Proposed threshold: {int(optimal_threshold_adc)} ADC")
plt.axhline(threshold_mv, color='b',label=f"Test threshold: {int(threshold_adc)} ")
plt.text(2500,430,f"baseline mean: {1000 * baseline_mean:.1f} mV")
plt.text(2500,400,f"baseline std: {1000 * baseline_std:.2f} mV")

plt.ylim(200,500)
plt.xlim(0,3900)
plt.xlabel("Time (ns)")
plt.ylabel("Voltage (mV)")
plt.legend()
plt.title(f"channel {channel}")
plt.show()

### Plot the area distribution

In [ ]:
areas = wfp.get_area(sum_window=(0.2,0.5))
plt.hist(areas,bins=200,range=(-0.1,1),histtype='step',color='red',label="Channel 0")
plt.ylabel("Count",fontsize=14)
plt.xlabel("Integrated Area [$V\cdot ns$]",fontsize=fontsize)
#plt.ylim(0,100)
plt.show()

In [ ]:
# Create a 2D array with the specified index sequence
array_2d = np.zeros((3, 8))
for col in range(8):
    array_2d[:, col] = [baseline_mean_list[col*3+1], baseline_mean_list[col*3], baseline_mean_list[col*3+2]]
# Plot the 2D array using colormap
plt.figure(figsize=(8, 3))
plt.imshow(array_2d, cmap='viridis', aspect='auto')
plt.colorbar(label='Baseline [mV]]')

# Add index labels to each cell
for i in range(3):
    for j in range(8):
        index = j * 3 + (1 if i == 0 else 0 if i == 1 else 2)
        plt.text(j, i, str(index), ha='center', va='center', color='r')



plt.xticks(range(8), range(1, 9))
plt.yticks(range(3), ['Top', 'Middle', 'Bottom'])
plt.title('Mean Baseline for each channel')
plt.savefig("baseline_mean.png")
plt.show()

In [ ]:
# Create a 2D array with the specified index sequence
array_2d = np.zeros((3, 8))
for col in range(8):
    array_2d[:, col] = [baseline_std_list[col*3+1], baseline_std_list[col*3], baseline_std_list[col*3+2]]
# Plot the 2D array using colormap
plt.figure(figsize=(8, 3))
plt.imshow(array_2d, cmap='viridis', aspect='auto')
plt.colorbar(label='RMS [mV]]')

# Add index labels to each cell
for i in range(3):
    for j in range(8):
        index = j * 3 + (1 if i == 0 else 0 if i == 1 else 2)
        plt.text(j, i, str(index), ha='center', va='center', color='r')



plt.xticks(range(8), range(1, 9))
plt.yticks(range(3), ['Top', 'Middle', 'Bottom'])
plt.title('Noise RMS per channel')
plt.savefig("baseline_std.png")
plt.show()